In [29]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [169]:
train_df = pd.read_csv('train.csv')

Deleting the first few columns based on the availability of data
Unsure of what features to drop for Garage. Garage cars vs Garage Quality.
How to deal with Garage Quality.


In [170]:
insufficient_data = ['Alley', 'Fence', 'MiscFeature', 'PoolQC', 'FireplaceQu', 
                     'LotFrontage']

imbalanced_data = ['3SsnPorch','BsmtCond', 'GarageQual', 'GarageCond', 'Heating', 'LandContour', 
                   'LandSlope', 'LowQualFinSF','PoolArea', 'RoofMatl', 'Street', 'Utilities']

nonsense_data = ['Condition1', 'Condition2', 'Functional', 'GarageType', 'GarageYrBlt', 
                 'BsmtHalfBath','LotConfig', 'LotShape', 'PavedDrive', 'MoSold', 'SaleCondition',
                 'YrSold']

extra_data = ['GarageCars', 'GarageFinish', 'HalfBath', 'BsmtFullBath', 'YearBuilt']


train_df.drop(insufficient_data + imbalanced_data + nonsense_data + extra_data, axis = 1,
             inplace = True)

In [171]:
train_df.shape
train_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Neighborhood', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'RoofStyle',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', 'ScreenPorch', 'MiscVal', 'SaleType', 'SalePrice'],
      dtype='object')

In [179]:
train_df['MasVnrType'].value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [178]:
# # train_df.isnull().sum() / train_df.shape[0] 
# train_df['PoolArea'].hist()
# plt.show()
# print(train_df['PoolArea'].describe())
# # train_df.shape

KeyError: 'PoolArea'

In [183]:
# train_df[train_df.columns[(train_df.isna().sum() / train_df.shape[0]) > 0]]
pd_series = train_df.isna().sum() / train_df.shape[0]
pd_series[pd_series > 0]
# train_df.columns

MasVnrType      0.005479
MasVnrArea      0.005479
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtFinType2    0.026027
Electrical      0.000685
dtype: float64

In [196]:
# train_df['BsmtExposure'] = train_df['BsmtExposure'].fillna('No Basement')

train_df[(train_df['BsmtFinType1'].isna() == False) & (train_df['BsmtFinType2'].isna() == True)][['BsmtFinType1', 'BsmtFinType2']]

,BsmtFinType1,BsmtFinType2
332,GLQ,NaN


In [137]:
# missing_vals = train_df[train_df.columns[train_df.isnull().sum() >0]]
# missing_vals
# # list(train_df.columns)
# to_drop = list(train_df.columns[train_df.isna().sum() > 300])
# train_df.drop(to_drop, axis =1, inplace = True)
# train_df[['GarageCars','GarageArea']]

,LotFrontage,Alley,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,NaN,BrkFace,196.0,Gd,TA,No,GLQ,Unf,SBrkr,NaN,Attchd,2003.0,RFn,TA,TA,NaN,NaN,NaN
1,80.0,NaN,None,0.0,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,1976.0,RFn,TA,TA,NaN,NaN,NaN
2,68.0,NaN,BrkFace,162.0,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,2001.0,RFn,TA,TA,NaN,NaN,NaN
3,60.0,NaN,None,0.0,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,1998.0,Unf,TA,TA,NaN,NaN,NaN
4,84.0,NaN,BrkFace,350.0,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,2000.0,RFn,TA,TA,NaN,NaN,NaN
5,85.0,NaN,None,0.0,Gd,TA,No,GLQ,Unf,SBrkr,NaN,Attchd,1993.0,Unf,TA,TA,NaN,MnPrv,Shed
6,75.0,NaN,Stone,186.0,Ex,TA,Av,GLQ,Unf,SBrkr,Gd,Attchd,2004.0,RFn,TA,TA,NaN,NaN,NaN
7,NaN,NaN,Stone,240.0,Gd,TA,Mn,ALQ,BLQ,SBrkr,TA,Attchd,1973.0,RFn,TA,TA,NaN,NaN,Shed
8,51.0,NaN,None,0.0,TA,TA,No,Unf,Unf,FuseF,TA,Detchd,1931.0,Unf,Fa,TA,NaN,NaN,NaN
9,50.0,NaN,None,0.0,TA,TA,No,GLQ,Unf,SBrkr,TA,Attchd,1939.0,RFn,Gd,TA,NaN,NaN,NaN


In [ ]:
# vars_to_use = ['MSSubClass','LotArea', 'LotFrontage', '']

# train_df[['LotFrontage','LotArea']].corr()